# Simulate sample sizes for batch-level comparison audits and ONEAudit

In [1]:
import math
import numpy as np
import scipy as sp
import pandas as pd
import os
import sys
import itertools

np.random.seed(seed=4197168)

In [2]:
def sprt_mart(x : np.array, N : int, mu : float=1/2, eta: float=1-np.finfo(float).eps, \
              u: float=1, random_order = True):
    '''
    Finds the p value for the hypothesis that the population 
    mean is less than or equal to mu against the alternative that it is eta,
    for a population of size N of values in the interval [0, u].
    
    Generalizes Wald's SPRT for the Bernoulli to sampling without replacement and to bounded
    values rather than binary values.

    If N is finite, assumes the sample is drawn without replacement
    If N is infinite, assumes the sample is with replacement
    
    Data are assumed to be in random order. If not, the calculation for sampling without replacement is incorrect.


    
    Parameters:
    -----------
    x : binary list, one element per draw. A list element is 1 if the 
        the corresponding trial was a success
    N : int
        population size for sampling without replacement, or np.infinity for 
        sampling with replacement
    theta : float in (0,u)
        hypothesized population mean
    eta : float in (0,u)
        alternative hypothesized population mean
    random_order : Boolean
        if the data are in random order, setting this to True can improve the power.
        If the data are not in random order, set to False
    '''
    if any((xx < 0 or xx > u) for xx in x):
        raise ValueError(f'Data out of range [0,{u}]')
    if np.isfinite(N):
        if not random_order:
            raise ValueError("data must be in random order for samples without replacement")
        S = np.insert(np.cumsum(x),0,0)[0:-1]  # 0, x_1, x_1+x_2, ...,  
        j = np.arange(1,len(x)+1)              # 1, 2, 3, ..., len(x)
        m = (N*mu-S)/(N-j+1)                   # mean of population after (j-1)st draw, if null is true
    else:
        m = mu
    with np.errstate(divide='ignore',invalid='ignore'): 
        terms = np.cumprod((x*eta/m + (u-x)*(u-eta)/(u-m))/u) # generalization of Bernoulli SPRT
    terms[m<0] = np.inf                        # the null is surely false
    return terms

In [3]:
def shrink_trunc(x: np.array, N: int, mu: float=1/2, nu: float=1-np.finfo(float).eps, u: float=1, c: float=1/2, 
                 d: float=100) -> np.array: 
    '''
    apply the shrinkage and truncation estimator to an array
    
    sample mean is shrunk towards nu, with relative weight d times the weight of a single observation.
    estimate is truncated above at u-u*eps and below at mu_j+e_j(c,j)
    
    S_1 = 0
    S_j = \sum_{i=1}^{j-1} x_i, j > 1
    m_j = (N*mu-S_j)/(N-j+1) if np.isfinite(N) else mu
    e_j = c/sqrt(d+j-1)
    eta_j =  ( (d*nu + S_j)/(d+j-1) \vee (m_j+e_j) ) \wedge u*(1-eps)
    
    Parameters
    ----------
    x : np.array
        input data       
    mu : float in (0, 1)
        hypothesized population mean
    eta : float in (t, 1)
        initial alternative hypothethesized value for the population mean
    c : positive float
        scale factor for allowing the estimated mean to approach t from above
    d : positive float
        relative weight of nu compared to an observation, in updating the alternative for each term
    '''
    S = np.insert(np.cumsum(x),0,0)[0:-1]  # 0, x_1, x_1+x_2, ...,  
    j = np.arange(1,len(x)+1)              # 1, 2, 3, ..., len(x)
    m = (N*mu-S)/(N-j+1) if np.isfinite(N) else mu   # mean of population after (j-1)st draw, if null is true 
    return np.minimum(u*(1-np.finfo(float).eps), np.maximum((d*nu+S)/(d+j-1),m+c/np.sqrt(d+j-1)))

In [4]:
def alpha_mart(x: np.array, N: int, mu: float=1/2, eta: float=1-np.finfo(float).eps, u: float=1, \
               estim: callable=shrink_trunc) -> np.array :
    '''
    Finds the ALPHA martingale for the hypothesis that the population 
    mean is less than or equal to t using a martingale method,
    for a population of size N, based on a series of draws x.
    
    The draws must be in random order, or the sequence is not a martingale under the null
    
    If N is finite, assumes the sample is drawn without replacement
    If N is infinite, assumes the sample is with replacement

    Parameters
    ----------
    x : list corresponding to the data
    N : int
        population size for sampling without replacement, or np.infinity for sampling with replacement
    mu : float in (0,1)
        hypothesized fraction of ones in the population
    eta : float in (t,1) 
        alternative hypothesized population mean
    estim : callable
        estim(x, N, mu, eta, u) -> np.array of length len(x), the sequence of values of eta_j for ALPHA
               
    Returns
    -------   
    terms : array
        sequence of terms that would be a nonnegative supermartingale under the null
    '''
    S = np.insert(np.cumsum(x),0,0)[0:-1]  # 0, x_1, x_1+x_2, ...,  
    j = np.arange(1,len(x)+1)              # 1, 2, 3, ..., len(x)
    m = (N*mu-S)/(N-j+1) if np.isfinite(N) else mu   # mean of population after (j-1)st draw, if null is true 
    etaj = estim(x, N, mu, eta, u) 
    with np.errstate(divide='ignore',invalid='ignore'):
        terms = np.cumprod((x*etaj/m + (u-x)*(u-etaj)/(u-m))/u)
    terms[m<0] = np.inf
    return terms

In [5]:
def oneaudit(x, alpha: float, u_over: float, eta: float, dl: list, c: float=1/2, reps=10**3, verbose=True):
    '''
    test whether the assorter mean is \le 1/2 using ballot polling, with ALPHA mart and the SPRT
    
    Parameters
    ----------
    x: numpy array
        the data
    alpha: float
        risk limit
    u_over: float
        a priori upper bound on the population of scaled overstatements
    eta: float
        starting alternative value 
    '''
    resl = ['rej_N','not_rej_N']

    results_inf = {}
    for r in resl:
        results_inf[r]=0

    results_d = {}
    for d in dl:
        results_d[d] = {}
        for r in resl:
            results_d[d][r]=0

    N = len(x)
    if verbose:
        print('reps: ', end=' ')
    for i in range(reps):
        if verbose:
            print(f'{i} ', end=' ')
        np.random.shuffle(x)                
        # OneAudit a priori
        mart = sprt_mart(x, N, mu=1/2, eta=eta, u=u_over, random_order=True)
        # rejections by N
        found = np.argmax(mart >= 1/alpha)
        results_inf['rej_N'] += found
        results_inf['not_rej_N'] += (found==0) # should not occur        

        # OneAudit ALPHA
        for d in dl:               
            mart = alpha_mart(x, N, mu=1/2, eta=eta, u=u_over, \
                              estim=lambda x, N, mu, eta, u: shrink_trunc(x,N,mu,eta,u,c=c,d=d))
            # rejections by N
            found = np.argmax(mart >= 1/alpha)
            results_d[d]['rej_N'] += found
            results_d[d]['not_rej_N'] += (found==0) # should not occur 
    for r in resl:
        results_inf[r] /= reps

    for d in dl:
        for r in resl:
            results_d[d][r] /= reps
    return results_inf, results_d

In [6]:
def ballot_polling(x, alpha: float, eta: float, dl: list, c: float=1/2, reps=10**3, verbose=True):
    '''
    test whether the assorter mean is \le 1/2 using ballot polling, with ALPHA mart and the SPRT
    '''
    resl = ['rej_N','not_rej_N']

    results_inf = {}
    for r in resl:
        results_inf[r]=0

    results_d = {}
    for d in dl:
        results_d[d] = {}
        for r in resl:
            results_d[d][r]=0
    N = len(x)

    if verbose:
        print('reps: ', end=' ')
    for i in range(reps):
        if verbose:
            print(f'{i} ', end=' ')
        np.random.shuffle(x)                
        # OneAudit a priori
        mart = sprt_mart(x, N, mu=1/2, eta=eta, u=1, random_order=True)
        # rejections by N
        found = np.argmax(mart >= 1/alpha)
        results_inf['rej_N'] += found
        results_inf['not_rej_N'] += (found==0) # should not occur        

        # OneAudit ALPHA
        for d in dl:               
            mart = alpha_mart(x, N, mu=1/2, eta=eta, u=1, \
                              estim=lambda x, N, mu, eta, u: shrink_trunc(x,N,mu,eta,u,c=c,d=d))
            # rejections by N
            found = np.argmax(mart >= 1/alpha)
            results_d[d]['rej_N'] += found
            results_d[d]['not_rej_N'] += (found==0) # should not occur    
    for r in resl:
        results_inf[r] /= reps

    for d in dl:
        for r in resl:
            results_d[d][r] /= reps
    return results_inf, results_d

In [7]:
def oneaudit_ppeb(x, alpha: float, u_over: float, eta: float, dl: list, c: float=1/2, reps=10**3, verbose=True):
    '''
    test whether the assorter mean is \le 1/2 using weighted sampling ballot polling with , with ALPHA mart and the SPRT
    
    Parameters
    ----------
    x: numpy array
        the data
    alpha: float
        risk limit
    u_over: float
        a priori upper bound on the population of scaled overstatements
    eta: float
        starting alternative value 
    '''
    resl = ['rej_N','not_rej_N']

    results_inf = {}
    for r in resl:
        results_inf[r]=0

    results_d = {}
    for d in dl:
        results_d[d] = {}
        for r in resl:
            results_d[d][r]=0

    N = len(x)
    if verbose:
        print('reps: ', end=' ')
    for i in range(reps):
        if verbose:
            print(f'{i} ', end=' ')
        np.random.shuffle(x)                
        # OneAudit a priori
        mart = sprt_mart(x, N, mu=1/2, eta=eta, u=u_over, random_order=True)
        # rejections by N
        found = np.argmax(mart >= 1/alpha)
        results_inf['rej_N'] += found
        results_inf['not_rej_N'] += (found==0) # should not occur        

        # OneAudit ALPHA
        for d in dl:               
            mart = alpha_mart(x, N, mu=1/2, eta=eta, u=u_over, \
                              estim=lambda x, N, mu, eta, u: shrink_trunc(x,N,mu,eta,u,c=c,d=d))
            # rejections by N
            found = np.argmax(mart >= 1/alpha)
            results_d[d]['rej_N'] += found
            results_d[d]['not_rej_N'] += (found==0) # should not occur 
    for r in resl:
        results_inf[r] /= reps

    for d in dl:
        for r in resl:
            results_d[d][r] /= reps
    return results_inf, results_d

In [8]:
def B(c: float, b: float, v: float, u: float=1):
    '''
    overstatement assorter
    
    Parameters
    ----------
    c: assorter applied to the CVR
    b: assorter applied to the physical card
    v: reported assorter margin, 2*(assorter mean)-1
    u: a priori upper bound on the assorter
    
    Returns
    -------
    overstatement assorter value    
    '''
    return (u+b-c)/(2*u-v)

In [9]:
def make_data(batches: pd.DataFrame, v: float, eta: float, u: float=1):
    '''
    make CVRs and assorter values for simulations
    re-scales the overstatement assorter across batches to have minimum 0
    
    Parameters
    ----------
    batches: pandas dataframe
        batch data
    v: float
        reported assorter margin
    eta: float
        alternative value for the overstatement assorter mean
    u: float
        upper bound on the raw assorter
        
    Returns
    -------
    tuple:
        synth: ONEAudit data
        synth_a: raw assorter data
        ub: upper bound on any of the scaled overstatements
        
    depends
    -------
    B: callable
        overstatement assorter
    winr: string
        column ID for winner
    losr: string
        column ID for loser
    cards: string
        column ID for number of cards
    '''
    synth = []
    synth_a = []
    lb, ub = 1, 0
    for ix, r in batches.iterrows():
        ub = max(ub, big := B(r['A'], 1, v, u))
        lb = min(lb, small := B(r['A'], 0, v, u))       
        synth.append(big*np.ones(r[winr]))
        synth.append(small*np.ones(r[losr]))
        synth.append(B(r['A'], 1/2, v, u)*np.ones(r[cards]-r[winr]-r[losr]))
        synth_a.append(np.ones(r[winr]))
        synth_a.append(np.zeros(r[losr]))
        synth_a.append(0.5*np.ones(r[cards]-r[winr]-r[losr]))
    # renormalize synth to have minimum 0 and null mean 1/2
    synth = np.array(list(itertools.chain(*synth)))
    scale = 1/(1-2*lb)  # restore null mean to 1/2
    synth = (synth-lb)*scale # shift minumum to 0
    u_synth = (ub-lb)*scale
    eta_synth = (eta-lb)*scale
    synth_a = np.array(list(itertools.chain(*synth_a)))
    return synth, synth_a, eta_synth, u_synth


# Synthetic AvB

Suppose there were 20,000 cards cast in the Alice v. Bob contest.
Of the 20,000 ballot cards, 10,000 were cast by mail and have linked CVRs.
Of those 10,000 CVRs, 5,000 report votes for Alice, 4,000 report votes for Bob, and 1,000 report undervotes.
The remaining 10,000 cards were cast in 10 precincts numbered 1 to 10, with 1,000 cards in each.
The batch subtotals for 5 of those precincts show 900 votes for Alice and 100 for Bob; the other 5 show 900 votes for Bob and
100 for Alice.
The reported results are thus 10,000 votes for Alice, 9,000 for Bob, and 1,000 undervotes.
The margin is 1,000 votes; the \emph{diluted margin} (margin in votes, divided by cards cast) is $1000/20000 = 5\%$.

In [10]:
N = 20000
Abar = (10000*1 + 1000*.5 + 9000*0)/N
v = 2*Abar-1
u=1
eta = u/(2*u-v)
u_over = 2*eta

In [11]:
print(f'{eta=} {v=} {u_over=}')

eta=0.5128205128205129 v=0.050000000000000044 u_over=1.0256410256410258


In [12]:
faa = 0.9 # fraction of votes for Alice in an Alice-majority precinct
fab = 0.1 # fraction of votes for Alice in an Bob-majority precinct
N_per_pct = 1000
pct = 10
# vote for Alice in a precinct with a majority for Alice:
aa = B(faa, 1, v, u)
# vote for Bob in a precinct with a majority for Alice:
ba = B(faa, 0, v, u)
# vote for Alice in a precinct with a majority for Bob:
ab = B(fab, 1, v, u) 
# vote for Bob in a precinct with a majority for Bob:
bb = B(fab, 0, v, u)
# CVRs
c = B(1, 1, v, u)
# make the overstatement data
x = np.array([c]*10000 
             + [aa]*int(faa*N_per_pct*pct/2)
             + [ba]*int((1-faa)*N_per_pct*pct/2)
             + [ab]*int(fab*N_per_pct*pct/2)
             + [bb]*int((1-fab)*N_per_pct*pct/2)
            )
print(f'{aa=} {ba=} {ab=} {bb=} {len(x)} {np.mean(x)}')

aa=0.5641025641025642 ba=0.05128205128205127 ab=0.9743589743589743 bb=0.46153846153846156 19999 0.5128435908974937


In [13]:
c = 1/2
reps = 10**4
dl = [10, 100, 1000, 10000]
alpha = 0.05

results_inf, results_d = oneaudit(x, alpha, u_over, u_over, dl, c, reps, verbose=False)
results_inf, results_d

({'rej_N': 8469.0914, 'not_rej_N': 0.0},
 {10: {'rej_N': 2697.6558, 'not_rej_N': 0.0},
  100: {'rej_N': 910.8872, 'not_rej_N': 0.0},
  1000: {'rej_N': 960.9379, 'not_rej_N': 0.0},
  10000: {'rej_N': 2943.1339, 'not_rej_N': 0.0}})

In [14]:
results_inf, results_d = ballot_polling(x, alpha, Abar, dl, c, reps, verbose=False)
results_inf, results_d

({'rej_N': 2232.1565, 'not_rej_N': 0.0},
 {10: {'rej_N': 2999.0083, 'not_rej_N': 0.0},
  100: {'rej_N': 3202.1575, 'not_rej_N': 0.0},
  1000: {'rej_N': 2908.4767, 'not_rej_N': 0.0},
  10000: {'rej_N': 2340.5868, 'not_rej_N': 0.0}})

## Kaplan-Markov batch-level comparison using PPEB


Convert to SHANGRLA notation. 

Ballots are partitioned into batches $\{\mathcal{B}_k\}_{k=1}^K$.
Batch $\mathcal{B}_k$ contains $N_k \ge 0$ cards; there are $N = \sum_k N_k$ cards in all.
The assorter upper bound is $u$.
The true assorter total for batch $k$ is $A_k^b \in [0, N_k u]$; the reported total for batch $k$
is $A_k^c \in [0, N_k u]$;
the _overstatement_ in batch $k$ is $E_k := A_k^c - A_k^b \in [A_k^c-Nu, A_k^c]$ and
the _taint_ in batch $k$ is $T_k := A_k^b/A_k^c \in  [0, N_ku/A_k^c]$.
The reported assorter total is $A_0^c := \sum_k A_k^c = N\bar{A}^c$;
the true assorter total is $A_0^b := \sum_k A_k^b = N\bar{A}^b$; the _total overstatement_ is $E_0 = A_0^c - A_0^b$;
and the _total taint_ is $T_0 := 1-E_0/A_0^c = A_0^b/A_0^c$.

The assertion is true if $\bar{A}^b > 1/2$, i.e., if $A_0^b > N/2$ or 
$A_0^b/A_0^c > (N/2)/A_0^c$. 

We sample batches, with probability of selecting batch $j$ proportional to $A_j^c$.
Let $J_i$ denote the index of the batch selected on the $j$th draw; let 
$T_i := A_{J_i}^b/A_{J_i}^c \ge 0$ be the taint of that batch.
If the sample is drawn without replacement, let $\mathcal{J}_i := \cup_{j=1}^i J_i$ be the 
indices of the batches that have been selected
up to and including the $i$th draw.
If the sample is drawn with replacement, let $\mathcal{J}_i := \emptyset$.
For any $\mathcal{J} \subset \{1, \ldots, K\}$, let
\begin{eqnarray}
\bar{\mathcal{J}} &:=& \{1, \ldots, K\} \setminus \mathcal{J} \mbox{ (complement) }\\
A_{\mathcal{J}}^b &:=& \sum_{j \in \mathcal{J}} A_j^b \\
A_{\mathcal{J}}^c &:=& \sum_{j \in \mathcal{J}} A_j^c.
\end{eqnarray}
The chance of drawing batch $j$ in draw $i$ is zero if $j \in \mathcal{J}_i$; it is
$A_j^c/(A_0^c-A_{\mathcal{J}_i}^c )$ if $j \notin \mathcal{J}_i$.
(For sampling with replacement, this is $A_j^c/A_0^c$ in every draw.)
Now
\begin{eqnarray}
\mathbb{E} (T_i | T_1, \ldots, T_{i-1}) 
   &=& \sum_{j \in \bar{\mathcal{J}}_{i-1}} \frac{A_j^c}{A_{\bar{\mathcal{J}}_{i-1}}^c} (A_j^b/A_j^c) \\
   &=&  \frac{A_{\bar{\mathcal{J}}_{i-1}}^b}{A_{\bar{\mathcal{J}}_{i-1}}^c}.
\end{eqnarray}
Suppose $A_0^b/A_0^c = \mu$, so $E = U(1-\mu)$.
Then $E_{\bar{\mathcal{J}}_{i-1}} = U(1-\mu) - E_{\mathcal{J}_{i-1}}$ and
$U_{\bar{\mathcal{J}}_{i-1}} = U - U_{\mathcal{J}_{i-1}}$.
\begin{eqnarray}
\mathbb{E} (T_i | T_1, \ldots, T_{i-1}) 
  &=& 1 - \frac{U(1-\mu) - E_{\mathcal{J}_{i-1}}}{U - U_{\mathcal{J}_{i-1}}} \\
  &=& \frac{\mu U + E_{\mathcal{J}_{i-1}} - U_{\mathcal{J}_{i-1}}}{U - U_{\mathcal{J}_{i-1}}}.
\end{eqnarray}
For sampling with replacement, that reduces to $\mu$ because $\mathcal{J}_{i-1} = \emptyset$.
Let $M_0 :=1$ and for $j>1$ define 
\begin{equation}
M_j := \prod_{i \le j} T_i \cdot \frac{U - U_{\mathcal{J}_{i-1}}}{N/2 + E_{\mathcal{J}_{i-1}} - U_{\mathcal{J}_{i-1}}}.
\end{equation}
Then $(M_j)_{j \in \mathbb{N}}$ is a nonnegative supermartingale starting at 1
if $1-E/U \le N/(2U)$. 

To estimate sample sizes for batch comparison audits, we will assume all $E_j$ are zero, so each term in the product contributes $2U/N$ to the supermartingale.
The number of draws for the test statistic to exceed $1/\alpha$ is $n = \lceil \log(1/\alpha)/\log(2U/N) \rceil$.
The chance batch $j$ is selected at least once in $n$ draws is $p_j := 1 - (1-U_j/U)^n$, so the
expected sample size is $\sum_j p_jN_j = \sum_j N_j(1 - (1-U_j/U)^n)$.

In [15]:
def km_batch(Ns: list, x: list, alpha: float=0.05) -> tuple:
    '''
    Kaplan-Markov draws and expected sample size for batch-comparison audit of one assorter, 
    no allowance for error.
    
    Parameters
    ----------
    Ns: list of ints
        number of ballots in each batch
    
    x: list of floats
        the assorter total for each batch
    
    alpha: float in (0, 1)
        risk limit
    
    Returns
    -------
    n: int
        minimum number of draws
    expect: float
        expected number of distinct ballots in the sample
    '''
    N = np.sum(Ns)
    U = np.sum(x)
    n = math.ceil(math.log(1/alpha)/math.log(2*U/N))
    expect = np.sum(Ns*(1-(1-x/U)**n))
    return n, expect

def batch_assort(Ns: list, winr: list, losr: list) -> np.array:
    '''
    turn batch subtotals for the winner and loser into assorter totals for the batch
    
    Parameters
    ----------
    Ns: list of ints
        batch sizes
    winr: list of ints
        votes for the winner in each batch
    losr: list of ints
        votes for the loser in each batch
    
    Returns
    -------
    tots: np.array of ints
        assorter totals for each batch
    '''
    w = np.array(winr)
    return (w + 0.5*(np.array(Ns) - w - np.array(losr)))

In [16]:
faa = 0.99 # fraction of votes for Alice in an Alice-majority precinct
fab = 0.01 # fraction of votes for Alice in an Bob-majority precinct
pct = 10

winr = np.array(
           [1]*5000 
           + [0]*5000 
           + [int(faa*N_per_pct)]*int(pct/2) 
           + [int(fab*N_per_pct)]*int(pct/2)
        )
losr = np.array(
           [0]*5000 + [1]*4000 + [0]*1000
         + [int((1-faa)*N_per_pct)]*int(pct/2)
         + [int((1-fab)*N_per_pct)]*int(pct/2)
        )
Ns = np.array([1]*10000 + [1000]*10)

x = batch_assort(Ns, winr, losr)
alpha = 0.05
km_batch(Ns, x, alpha)

(62, 5308.434319418066)

# Georgia 2022 audit

In [17]:
file_dir = './2022-11-08-georgia-rla-county-batch-tallies' # 2022 Georgia midterms
col_names = ['Batch Name', 'Brad Raffensperger (I) (Rep)', 'Bee Nguyen (Dem)', 'Ted Metz (Lib)']

winr = col_names[1]
losr = col_names[2]
othr = col_names[3]
cards = 'cards'

In [18]:
first = True
rows = 0
n_files = 0
for subdir, dirs, files in os.walk(file_dir):
    for file in files:
        if file.endswith('.csv'):
            n_files += 1
            batch = pd.read_csv(os.path.join(subdir,file), sep=',', header=0, names=col_names)
            rows += len(batch[col_names[0]])
            if first: 
                first = False
                batches = batch.copy()
            else:
                batches = pd.concat([batches, batch])

print(f'{n_files=}')

n_files=159


In [19]:
len(batches['Batch Name']), rows

(12968, 12968)

In [20]:
batches.columns

Index(['Batch Name', 'Brad Raffensperger (I) (Rep)', 'Bee Nguyen (Dem)',
       'Ted Metz (Lib)'],
      dtype='object')

In [21]:
batches.head()

,Batch Name,Brad Raffensperger (I) (Rep),Bee Nguyen (Dem),Ted Metz (Lib)
0,ICC - Absentee By Mail - 1,26,22,0
1,ICC - Absentee By Mail - 4,31,11,1
2,ICC - Absentee By Mail - 5,16,33,0
3,ICC - Absentee By Mail - 6,28,20,1
4,ICC - Absentee By Mail - 7,32,17,0


In [22]:
# reported results per https://results.enr.clarityelections.com/GA/115465/web.307039/#/summary, 12/15/22
results = [2081241, 1719922, 108884]
batch_results = {}

for i, cn in enumerate(col_names[1:]):
    batch_results[cn] = np.sum(batches[cn])
    print(f'{cn}: {batch_results[cn]} \t reported: {results[i]} \t diff: {batch_results[cn]-results[i]}')

Brad Raffensperger (I) (Rep): 2081211 	 reported: 2081241 	 diff: -30
Bee Nguyen (Dem): 1719889 	 reported: 1719922 	 diff: -33
Ted Metz (Lib): 108883 	 reported: 108884 	 diff: -1


In [23]:
batches[cards] = batches[winr] + batches[losr] + batches[othr]  # the batch reports do not include the number of cards
n_cards = np.sum(batches[cards])

In [24]:
# diluted margin
(2081211-1719922)/n_cards

0.09240168051881556

In [25]:
n_cards

3909983

In [26]:
# sampling probabilities
batches['p'] = batches[cards]/n_cards

# reported assorter means
batches['A'] = (batches[winr] + 0.5*(batches[cards]-batches[winr]-batches[losr]))/batches[cards]

# reported overall assorter mean
Abar = np.sum(batches['A']*batches[cards])/n_cards

margin_votes = np.sum(batches[winr]) - np.sum(batches[losr])
v = 2*Abar-1
u=1
eta = u/(2*u-v)
u_over = 2*eta

print(f'{n_cards=} {margin_votes=} {v= :.4f} {eta= :.4f} {u_over= :.4f}')

n_cards=3909983 margin_votes=361322 v= 0.0924 eta= 0.5242 u_over= 1.0484


In [27]:
# make synthetic set of assorter values and corresponding overstatements
synth, synth_a, eta_synth, u_synth = make_data(batches, v, eta, u)

len(synth), len(synth_a), n_cards, eta_synth, u_synth

(3909983, 3909983, 3909983, 0.5242216949890623, 1.0484433899781247)

In [28]:
c = 1/2
reps = 10**2
dl = [10, 100, 1000]
alpha = 0.05

In [29]:
x = synth.copy()
results_inf, results_d = oneaudit(x, alpha, u_synth, eta_synth, dl, c, reps)
results_inf, results_d

reps:  0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99  

({'rej_N': 1504.1, 'not_rej_N': 0.0},
 {10: {'rej_N': 1363.3, 'not_rej_N': 0.0},
  100: {'rej_N': 1465.64, 'not_rej_N': 0.0},
  1000: {'rej_N': 1567.02, 'not_rej_N': 0.0}})

In [30]:
x = synth_a.copy()
results_inf, results_d = ballot_polling(x, alpha, Abar, dl, c, reps)
results_inf, results_d

reps:  0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99  

({'rej_N': 687.47, 'not_rej_N': 0.0},
 {10: {'rej_N': 870.41, 'not_rej_N': 0.0},
  100: {'rej_N': 761.35, 'not_rej_N': 0.0},
  1000: {'rej_N': 702.25, 'not_rej_N': 0.0}})

In [31]:
len(batches[cards]), len(batches[winr])

(12968, 12968)

In [32]:
x = batch_assort(batches[cards], batches[winr], batches[losr])
alpha = 0.05
km_batch(batches[cards], x, alpha)

(34, 103287.04134915333)

# California 2020

Data from https://statewidedatabase.org/d10/g20.html

## President

In [33]:
fn = './CA/state_g20_sov_data_by_g20_srprec.csv'
batches_ca_20 = pd.read_csv(fn)
batches = batches_ca_20.copy()

In [34]:
col_names = batches.columns
col_names

Index(['COUNTY', 'FIPS', 'SRPREC_KEY', 'SRPREC', 'ADDIST', 'CDDIST', 'SDDIST',
       'BEDIST', 'TOTREG', 'DEMREG', 'REPREG', 'AIPREG', 'GRNREG', 'LIBREG',
       'NLPREG', 'REFREG', 'DCLREG', 'MSCREG', 'TOTVOTE', 'DEMVOTE', 'REPVOTE',
       'AIPVOTE', 'GRNVOTE', 'LIBVOTE', 'NLPVOTE', 'REFVOTE', 'DCLVOTE',
       'MSCVOTE', 'PRCVOTE', 'ABSVOTE', 'ASSDEM01', 'ASSDEM02', 'ASSGRN01',
       'ASSIND01', 'ASSLIB01', 'ASSREP01', 'ASSREP02', 'CNGDEM01', 'CNGDEM02',
       'CNGREP01', 'PRSAIP01', 'PRSDEM01', 'PRSGRN01', 'PRSLIB01', 'PRSPAF01',
       'PRSREP01', 'PR_14_N', 'PR_14_Y', 'PR_15_N', 'PR_15_Y', 'PR_16_N',
       'PR_16_Y', 'PR_17_N', 'PR_17_Y', 'PR_18_N', 'PR_18_Y', 'PR_19_N',
       'PR_19_Y', 'PR_20_N', 'PR_20_Y', 'PR_21_N', 'PR_21_Y', 'PR_22_N',
       'PR_22_Y', 'PR_23_N', 'PR_23_Y', 'PR_24_N', 'PR_24_Y', 'PR_25_N',
       'PR_25_Y', 'SENAIP01', 'SENDEM01', 'SENDEM02', 'SENLIB01', 'SENREP01'],
      dtype='object')

In [35]:
len(batches[col_names[0]])

21346

In [36]:
for i, cn in enumerate(col_names[18:]):
    print(f'{i+18} {cn} {np.sum(batches[cn])}')

18 TOTVOTE 17785667
19 DEMVOTE 0
20 REPVOTE 0
21 AIPVOTE 0
22 GRNVOTE 0
23 LIBVOTE 0
24 NLPVOTE 0
25 REFVOTE 0
26 DCLVOTE 0
27 MSCVOTE 0
28 PRCVOTE 0
29 ABSVOTE 0
30 ASSDEM01 9133718
31 ASSDEM02 945914
32 ASSGRN01 41100
33 ASSIND01 157133
34 ASSLIB01 76377
35 ASSREP01 5588332
36 ASSREP02 133825
37 CNGDEM01 10156664
38 CNGDEM02 927963
39 CNGREP01 5640768
40 PRSAIP01 60162
41 PRSDEM01 11110639
42 PRSGRN01 81032
43 PRSLIB01 187910
44 PRSPAF01 51038
45 PRSREP01 6006518
46 PR_14_N 8222302
47 PR_14_Y 8588954
48 PR_15_N 8885734
49 PR_15_Y 8213383
50 PR_16_N 9655878
51 PR_16_Y 7217278
52 PR_17_N 7069286
53 PR_17_Y 9985952
54 PR_18_N 9578025
55 PR_18_Y 7514596
56 PR_19_N 8176289
57 PR_19_Y 8546103
58 PR_20_N 10294406
59 PR_20_Y 6385965
60 PR_21_N 10095407
61 PR_21_Y 6771578
62 PR_22_N 7028077
63 PR_22_Y 9958669
64 PR_23_N 10681400
65 PR_23_Y 6161704
66 PR_24_N 7305584
67 PR_24_Y 9384955
68 PR_25_N 9358419
69 PR_25_Y 7232660
70 SENAIP01 89080
71 SENDEM01 5056543
72 SENDEM02 594590
73 SENLIB01 53

In [37]:
winr = col_names[41]
losr = col_names[45]
cards = col_names[18]

n_cards = np.sum(batches[cards])
# sampling probabilities
batches['p'] = batches[cards]/n_cards

In [38]:
# Assorter means
batches['A'] = (batches[winr]+0.5*(batches[cards]-batches[winr]-batches[losr]))/batches[cards]

# Reported overall assorter mean
Abar = np.sum(batches['A']*batches[cards])/n_cards

margin_votes = np.sum(batches[winr]) - np.sum(batches[losr])
v = 2*Abar-1
u=1
eta = u/(2*u-v)
u_over = 2*eta

print(f'{n_cards=} {margin_votes=} {v= :.4f} {eta= :.4f} {u_over= :.4f}')

n_cards=17785667 margin_votes=5104121 v= 0.2870 eta= 0.5838 u_over= 1.1675


In [39]:
# make synthetic set of assorter values and corresponding overstatements

synth, synth_a, eta_synth, u_synth = make_data(batches, v, u)
len(synth), len(synth_a), n_cards, eta_synth, u_synth

(17785667, 17785667, 17785667, 1.0, 1.1675283197055142)

In [40]:
min(synth), max(synth)

(0.016635766213420534, 1.1548377944913237)

In [41]:
min(synth_a), max(synth_a)

(0.0, 1.0)

In [42]:
c = 1/2
reps = 10**3
dl = [10, 100, 1000]
alpha = 0.05

In [43]:
x = synth.copy()
results_inf, results_d = oneaudit(x, alpha, u_synth, eta_synth, dl, c, reps, verbost=False)
results_inf, results_d

TypeError: oneaudit() got an unexpected keyword argument 'verbost'

In [ ]:
x = synth_a.copy()
results_inf, results_d = ballot_polling(x, alpha, Abar, dl, c, reps, verbose=False)
results_inf, results_d

In [ ]:
batches.head()

In [ ]:
## batch-comparison
x = batch_assort(batches[cards], batches[winr], batches[losr])
alpha = 0.05
km_batch(batches[cards], x, alpha)

In [ ]:
26687/70

In [ ]:
np.sort(batches[cards])[-30:]

# 2020 California Prop 14

## Stem Cell Research Institute Bond Initiative

Official results

Yes 8,588,618 51.09%
No 8,222,154 48.91%

In [ ]:
batches = batches_ca_20.copy()

winr = col_names[47]
losr = col_names[46]
cards = col_names[18]

n_cards = np.sum(batches[cards])
# sampling probabilities
batches['p'] = batches[cards]/n_cards

In [ ]:
# there's a batch with more votes than ballots
test = list((batches[cards] - batches[winr] - batches[losr]) < 0)     
inx = test.index(1)

In [ ]:
row = batches.iloc[inx]
print(row)

In [ ]:
row[winr], row[losr], row[cards]

In [ ]:
# drop that batch of ballots
batches = batches.drop([inx])
n_cards = np.sum(batches[cards])
# re-compute sampling probabilities
batches['p'] = batches[cards]/n_cards

In [ ]:
# sampling probabilities
batches['p'] = batches[cards]/n_cards

# Assorter means
batches['A'] = (batches[winr]+0.5*(batches[cards]-batches[winr]-batches[losr]))/batches[cards]

# Reported overall assorter mean
Abar = np.sum(batches['A']*batches[cards])/n_cards

margin_votes = np.sum(batches[winr]) - np.sum(batches[losr])
v = 2*Abar-1
u=1
eta = u/(2*u-v)
u_over = 2*eta

print(f'{n_cards=} {margin_votes=} {v= :.4f} {eta= :.4f} {u_over= :.4f}')

In [ ]:
# make synthetic set of assorter values and corresponding overstatements

synth, synth_a, eta_synth, u_synth = make_data(batches, v, u)
len(synth), len(synth_a), n_cards, eta_synth, u_synth

In [ ]:
n_cards = 17785664
min(synth), max(synth)

In [ ]:
min(synth_a), max(synth_a)

In [ ]:
c = 1/2
reps = 10**2
dl = [10, 100, 1000]
alpha = 0.05

In [ ]:
x = synth.copy()
results_inf, results_d = oneaudit(x, alpha, u_synth, eta_synth, dl, c, reps)
results_inf, results_d

In [ ]:
x = synth_a.copy()
results_inf, results_d = ballot_polling(x, alpha, Abar, dl, c, reps)
results_inf, results_d